Notebook to examined 3.6 test runs

In [1]:
import netCDF4 as nc
import numpy as np
from salishsea_tools import nc_tools
import datetime
import os

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
path = '/data/nsoontie/MEOPAR/SalishSea/results/2Ddomain/3.6'
directory = 'densiity_output'
file_part = 'SalishSea_4h_20020915_20021012_{}.nc'

dT = nc.Dataset(os.path.join(path,directory,file_part.format('grid_T')))
sal = dT.variables['vosaline'][:]
sal = np.ma.masked_values(sal,0)
deps = dT.variables['deptht'][:]
temp = dT.variables['votemper'][:]
temp = np.ma.masked_values(temp,0)
ssh = dT.variables['sossheig'][:]
n2 = dT.variables['buoy_n2'][:]
n2 = np.ma.masked_values(n2,0)
times = dT.variables['time_counter'][:]
time_origin = datetime.datetime.strptime(dT.variables['time_counter'].time_origin, '%Y-%m-%d %H:%M:%S')

dU = nc.Dataset(os.path.join(path,directory,file_part.format('grid_U')))
U = dU.variables['vozocrtx'][:]
U = np.ma.masked_values(U,0)
depsU=dU.variables['depthu'][:]

dW = nc.Dataset(os.path.join(path,directory,file_part.format('grid_W')))
avt = dW.variables['vert_eddy_diff'][:]
avt = np.ma.masked_values(avt,0)
avm = dW.variables['vert_eddy_visc'][:]
avm = np.ma.masked_values(avm,0)
depsW=dW.variables['depthw'][:]
diss = dW.variables['dissipation'][:]
diss = np.ma.masked_values(diss,0)




RuntimeError: No such file or directory

#Salinity

In [ ]:
smin=22
smax=34
t=0
y=3
plt.pcolormesh(np.arange(0,sal.shape[3]),deps,sal[t,:,y,:],vmin=smin,vmax=smax)
plt.axis([0,1100,400,0])
date =time_origin + datetime.timedelta(seconds = times[t])
plt.title(date.strftime('%Y-%m-%d %H:%M'))
plt.colorbar()

In [ ]:
t=-1
plt.pcolormesh(np.arange(0,sal.shape[3]),deps,sal[t,:,y,:],vmin=smin,vmax=smax)
plt.axis([0,1100,400,0])
plt.colorbar()
date =time_origin + datetime.timedelta(seconds = times[t])
plt.title(date.strftime('%Y-%m-%d %H:%M'))

winds stress still too strong? Surface layer is quite deep.

#Temperature

In [ ]:
tmin=3; tmax=10
t=0
plt.pcolormesh(np.arange(0,sal.shape[3]),deps,temp[t,:,y,:],vmin=tmin,vmax=tmax)
plt.axis([0,1100,400,0])
plt.colorbar()
date =time_origin + datetime.timedelta(seconds = times[t])
plt.title(date.strftime('%Y-%m-%d %H:%M'))

In [ ]:

t=-1
plt.pcolormesh(np.arange(0,sal.shape[3]),deps,temp[t,:,y,:],vmin=tmin,vmax=tmax)
plt.axis([0,1100,400,0])
plt.colorbar()
date =time_origin + datetime.timedelta(seconds = times[t])
plt.title(date.strftime('%Y-%m-%d %H:%M'))

May want to be more careful about how the water temperature of the river is specified. 

# U

In [ ]:
t=0
umin=-1.5
umax=1.5
plt.pcolormesh(np.arange(0,U.shape[3]),depsU,U[t,:,y,:],vmin=umin,vmax=umax)
plt.axis([0,1100,400,0])
plt.colorbar()
date =time_origin + datetime.timedelta(seconds = times[t])
plt.title(date.strftime('%Y-%m-%d %H:%M'))

In [ ]:
t=-1
plt.pcolormesh(np.arange(0,U.shape[3]),depsU,U[t,:,y,:],vmin=umin,vmax=umax)
plt.axis([0,1100,400,0])
plt.colorbar()
date =time_origin + datetime.timedelta(seconds = times[t])
plt.title(date.strftime('%Y-%m-%d %H:%M'))

#SSH and tides

In [ ]:
dates = [time_origin + datetime.timedelta(seconds= t) for t in times]
plt.plot(ssh[:,y,10])
fig=plt.gcf()
fig.autofmt_xdate()
plt.grid()

Spring tides at t=60-70, 145-155

Amphidrome?

In [ ]:
plt.plot(dates,ssh[:,y,150])
fig=plt.gcf()
fig.autofmt_xdate()
plt.grid()

Yes, it looks like the tidal signal is mainly diurnal here at x=150.

In [ ]:
plt.pcolormesh(ssh[:,y,:],vmin=-4,vmax=4)
plt.colorbar()

In [ ]:
plt.plot(U[:,0,y,10])
plt.xlabel('U [m/s]')

In [ ]:
print U.max()
print U.min()

In [ ]:
plt.pcolormesh(U[:,0,y,:],vmin=-1.5,vmax=1.5)
plt.colorbar()

#Plot at a time

In [ ]:
def plot_variables(t):
    """Plot salinity, temperature, vertical eddy viscosity, vertical eddy diffusivity,
    buoyancy frequency, and U velocity at a chosen time. 
    
    t is an integer for model output index"""
    date =time_origin + datetime.timedelta(seconds = times[t])
    print date.strftime('%Y-%m-%d %H:%M')
    fig,axs = plt.subplots(3,2,figsize= (10,10))

    #salnity
    ax=axs[0,0]
    mesh=ax.pcolormesh(np.arange(0,sal.shape[3]),deps,sal[t,:,y,:],vmin=smin,vmax=smax)
    ax.set_ylim([400,0])
    ax.set_xlim([0,1100])
    plt.colorbar(mesh,ax=ax)
    ax.set_title('Salinity')

    #temp
    ax=axs[0,1]
    mesh=ax.pcolormesh(np.arange(0,temp.shape[3]),deps,temp[t,:,y,:],vmin=tmin,vmax=tmax)
    ax.set_ylim([400,0])
    ax.set_xlim([0,1100])
    plt.colorbar(mesh,ax=ax)
    ax.set_title('Temperature')

    #diff
    dmin=0; dmax=100
    ax=axs[1,1]
    mesh=ax.pcolormesh(np.arange(0,avt.shape[3]),depsW,avt[t,:,y,:],vmin=dmin, vmax=dmax, cmap='hot')
    ax.set_ylim([400,0])
    ax.set_xlim([0,1100])
    plt.colorbar(mesh,ax=ax)
    ax.set_title('Vertical eddy diff')

    #visct
    ax=axs[1,0]
    mesh=ax.pcolormesh(np.arange(0,avm.shape[3]),depsW,avm[t,:,y,:],vmin=dmin, vmax=dmax,cmap='hot')
    ax.set_ylim([400,0])
    ax.set_xlim([0,1100])
    plt.colorbar(mesh,ax=ax)
    ax.set_title('Vertical eddy visc')

    #n2
    nmin=0; nmax=0.005
    ax=axs[2,0]
    mesh=ax.pcolormesh(np.arange(0,n2.shape[3]),deps,n2[t,:,y,:],vmin=nmin, vmax=nmax)
    ax.set_ylim([400,0])
    ax.set_xlim([0,1100])
    plt.colorbar(mesh,ax=ax)
    ax.set_title('Squared buoyancy frequency')

    #U
    ax=axs[2,1]
    mesh=ax.pcolormesh(np.arange(0,U.shape[3]),depsU,U[t,:,y,:],vmin=umin, vmax=umax)
    ax.set_ylim([400,0])
    ax.set_xlim([0,1100])
    plt.colorbar(mesh,ax=ax)
    ax.set_title('U')
    
    return fig

Spring tide

In [ ]:
fig = plot_variables(67)

In [ ]:
fig = plot_variables(68)

Still not a lot of mixing over the sill. Is it too stratified? Is my second sill large enough?

Second spring

In [ ]:
fig = plot_variables(129)

Might need to be careful when masking the buoyancy frequency

In [ ]:
np.nanmax(avm)

Neap

In [ ]:
fig = plot_variables(110)

#Depth and thickness of cells

In [ ]:
print dT

In [ ]:
Tdep = dT.variables['T-depth_vvl']
Tthick = dT.variables['T-thick_vvl']

In [ ]:
t=75
plt.pcolormesh(Tthick[t,:,y,:])
plt.colorbar()
plt.axis([1000,1100,30,0])

This is showing the deformation of the T grid cell thickness due to the vvl scheme. At the far right, the cells are deformed the most. I think this is where the ssh amlpitudes are highest. 

In [ ]:
print Tthick

Cell depths - accounting for partial steps (I think)

In [ ]:
t=120
plt.pcolormesh(Tdep[t,:,y,:])
plt.colorbar()
plt.axis([0,1100,40,25])

How do these changes compare with the depth recorded in the depths variable?

In [ ]:
plt.plot(Tdep[t,-1,y,:])
plt.plot([0,1100],[deps[-1],deps[-1]], 'r-')

So there can be as much as a 30m difference between the bottom depth and the partial cell depth. How does that work? The grid spacing is ~27 m...

In [ ]:
print Tdep

grid spacing

In [ ]:
print deps[1:]-deps[:-1]

#Buoyancy frequency

Develop some code for analyzing stratification.

Goal: Define a Froude number to compare with 3D case. Trying to figure out how I can make the dynamics of my 2D case more like the 3D. 

Ultimately I will need density and currents and will need to relate these to the input somehow.


In [ ]:
print np.max(n2)
print np.unravel_index(np.argmax(n2), n2.shape)

Boundary forcing makes a strong density gradient near the start of the simulation.

In [ ]:
print np.min(n2)
print np.unravel_index(np.argmin(n2), n2.shape)

Unstable stratification

# Lats and Lons
What do nav_lat and nav_lon look like? 

In [ ]:
lat = dT.variables['nav_lat']
lon = dT.variables['nav_lon']

In [ ]:
lat

In [ ]:
lat[:]

In [ ]:
lon

In [ ]:
lon[:]

I don't like that they have units degress. That is weird to me.